## CX4240 project

## Classification of Acute Lymphoblastic Leukemia (ALL) in Blood Cell Images Using Machine Learning

# Dimension Reduction

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import csv 
import matplotlib as mpl
import plotly 
plotly.tools.set_credentials_file(username='JC-Chun', api_key='WXQ9jdGdWhUznzK2PZ50')
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls

from IPython.display import IFrame
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

%matplotlib inline


## Load Images

In [2]:
train_df = pd.read_csv('Data/train_data.csv')
train = pd.DataFrame.as_matrix(train_df)
test = pd.DataFrame.as_matrix(pd.read_csv('Data/test_data.csv'))
train_data = train[:,2:]
label_train = train[:,1].astype(int)
test_data = test[:,2:]
label_test = test[:,1].astype(int)
features = list(train_df.columns[2:]) 
print(np.shape(train_data))
print(np.shape(test_data))
n_ALL = 7272
n_hem = 3389
D = len(features)
features

(10661, 38)
(1867, 38)


C:\Users\1000j\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

C:\Users\1000j\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



['Cell Size',
 'Perimeter',
 'Form Factor',
 'Roundness',
 'Length/Diameter Ratio',
 'Compactness',
 'Boundary Roughness Variance',
 'Boundary Roughness Skewness',
 'Boundary Roughness Kurtosis',
 'Haralick Angular Second Moment',
 'Haralick Contrast',
 'Haralick Correlation',
 'Haralick Variance',
 'Haralick Inverse Difference Moment',
 'Haralick Sum Average',
 'Haralick Sum Variance',
 'Haralick Sum Entropy',
 'Haralick Entropy',
 'Haralick Difference Variance',
 'Haralick Difference Entropy',
 'Haralick Information Measures of Correlation 1',
 'Haralick Information Measures of Correlation 2',
 'Wavelet Approximation Mean',
 'Wavelet Horizontal Mean',
 'Wavelet Vertical Mean',
 'Wavelet Diagonal Mean',
 'Wavelet Approximation Variance',
 'Wavelet Horizontal Variance',
 'Wavelet Vertical Variance',
 'Wavelet Diagonal Variance',
 'Red Mean',
 'Green Mean',
 'Blue Mean',
 'Hue Mean',
 'Saturation Mean',
 'Value Mean',
 'Intensity Mean',
 'Intensity Variance']

## Dimension Reduction Using Random Forest

Feature ranking:
1. F1: Cell Size (0.150549)
2. F2: Perimeter (0.102639)
3. F20: Haralick Difference Entropy (0.033431)
4. F11: Haralick Contrast (0.031420)
5. F31: Red Mean (0.030748)
6. F36: Value Mean (0.029212)
7. F21: Haralick Information Measures of Correlation 1 (0.027698)
8. F22: Haralick Information Measures of Correlation 2 (0.027499)
9. F34: Hue Mean (0.027196)
10. F35: Saturation Mean (0.025771)
11. F15: Haralick Sum Average (0.024550)
12. F33: Blue Mean (0.024202)
13. F12: Haralick Correlation (0.023551)
14. F3: Form Factor (0.022161)
15. F37: Intensity Mean (0.022160)
16. F17: Haralick Sum Entropy (0.021987)
17. F13: Haralick Variance (0.021358)
18. F32: Green Mean (0.021154)
19. F38: Intensity Variance (0.021133)
20. F16: Haralick Sum Variance (0.019660)
21. F14: Haralick Inverse Difference Moment (0.019326)
22. F10: Haralick Angular Second Moment (0.019174)
23. F18: Haralick Entropy (0.019162)
24. F23: Wavelet Approximation Mean (0.018641)
25. F19: Haralick Difference V

The number of components:  38


In [3]:
# Finding the most important features using Random Forest
forest = ExtraTreesClassifier(n_estimators=100, random_state=0)
label_train = np.concatenate(([1]*n_ALL, [0]*n_hem), axis=0) # labels: 1=ALL  0=hem
forest.fit(train_data, label_train)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for i in range(train_data.shape[1]):
    print("%d. F%d: %s (%f)" % (i + 1, indices[i]+1, features[indices[i]], importances[indices[i]]))

# Plot the feature importances of the forest
plt.figure(figsize=(10,8))
plt.title("Feature Importances")
plt.bar(range(train_data.shape[1]), importances[indices],
        color="royalblue", yerr=std[indices], align="center")
plt.xticks(range(train_data.shape[1]), indices)
plt.xlim([-1, train_data.shape[1]])
plt.show()

# Find the number of components
cum_importance = importances[indices].cumsum()/importances.sum()
n_components = np.sum(cum_importance < .99) + 1
print("The number of components: ", n_components)

In [4]:
# Reduce train data to the first 16 important features
n_features = 16
RF_train_data = train_data[:,indices[:n_features]]

# Conversion of data from numpy array to dataframe
RF_train_data_df = pd.DataFrame.from_records(RF_train_data)
RF_train_data_df.columns = tuple(features[i] for i in indices[:n_features])
label_train = np.concatenate(([1]*n_ALL,[0]*n_hem), axis=0) 
RF_train_data_df.insert(0,"Label",label_train,True)
RF_train_data_df

# save train_data_df as a csv file
RF_train_data_df.to_csv('Data/RF_train_data')
RF_train_data_df

,Label,Cell Size,Perimeter,Haralick Difference Entropy,Haralick Contrast,Red Mean,Value Mean,Haralick Information Measures of Correlation 1,Haralick Information Measures of Correlation 2,Hue Mean,Saturation Mean,Haralick Sum Average,Blue Mean,Haralick Correlation,Form Factor,Intensity Mean,Haralick Sum Entropy
0,1,0.837601,0.314550,0.261523,0.167975,0.361517,0.359597,0.186455,0.924678,0.491460,0.829459,0.269314,0.362983,0.720569,0.809972,0.270446,0.460494
1,1,0.587318,0.359125,0.401331,0.376405,0.390337,0.384699,0.340154,0.893701,0.327081,0.837962,0.258010,0.308679,0.650502,0.435974,0.258415,0.472207
2,1,0.785577,0.370900,0.365521,0.160268,0.501905,0.518648,0.350370,0.920285,0.521127,0.442155,0.462431,0.482968,0.809072,0.586198,0.465471,0.599418
3,1,0.451474,0.186712,0.394068,0.235165,0.318079,0.311888,0.556612,0.622778,0.309685,0.871903,0.189900,0.235906,0.533764,0.814064,0.190651,0.191534
4,1,0.608631,0.225399,0.695308,0.401297,0.421780,0.415043,0.866907,0.490289,0.252923,0.788306,0.269031,0.291922,0.425492,0.886275,0.270343,0.393530
5,1,0.697439,0.254836,0.617579,0.361897,0.684556,0.680048,0.492463,0.901605,0.340460,0.585158,0.536790,0.561059,0.704162,0.879546,0.537420,0.697929
6,1,0.439167,0.216148,0.346074,0.247791,0.548840,0.543420,0.192713,0.963249,0.395196,0.669286,0.426224,0.473890,0.765314,0.671028,0.425316,0.669124
7,1,0.541160,0.195963,0.413293,0.246263,0.405301,0.399525,0.533481,0.707917,0.379973,0.840872,0.271772,0.341058,0.538670,0.921307,0.272992,0.295565
8,1,0.472464,0.179983,0.471991,0.304351,0.357267,0.352179,0.510594,0.774206,0.292658,0.776505,0.236912,0.268321,0.533411,0.883196,0.237908,0.376645
9,1,0.651026,0.274180,0.283365,0.260237,0.502251,0.494380,0.407362,0.808408,0.240214,0.820289,0.316716,0.343067,0.644882,0.744358,0.317154,0.329627


In [ ]:
# Data visualization
sns.set(style="ticks", color_codes=True)
g = sns.pairplot(RF_train_data_df,hue="Label")

Text(0.5, 0, 'Haralick Difference Entropy')

In [5]:
# Visualizing 3-D numeric data with Scatter Plots
# length, breadth and depth
df = RF_train_data_df.sample(frac=1)
fig = plt.figure(figsize=(16, 10))
ax = plt.axes(projection='3d')
xs = list(RF_train_data_df['Cell Size'])
ys = list(RF_train_data_df['Perimeter'])
zs = list(RF_train_data_df['Haralick Difference Entropy'])
data_points = [(x, y, z) for x, y, z in zip(xs, ys, zs)]
colors = ['orange' if i == 1 else 'blue' for i in list(RF_train_data_df['Label'])]
for data, color in zip(data_points, colors):
    x, y, z = data
    ax.scatter(x, y, z,s=30, alpha=0.4, c=color, edgecolors='w')

ax.set_xlabel('Cell Size')
ax.set_ylabel('Perimeter')
ax.set_zlabel('Haralick Difference Entropy')

In [ ]:
## Visualization of top 3 features
# Trace of ALL
x = RF_train_data_df['Cell Size']
y = RF_train_data_df['Perimeter']
z = RF_train_data_df['Haralick Difference Entropy']
ALL = go.Scatter3d(
        x = x[:n_ALL],
        y = y[:n_ALL],
        z = z[:n_ALL],
        opacity = 0.5,
        name = 'ALL',
        mode = 'markers',
        marker = dict(
            size = 5,
            color = 'rgb(255,102,102)',
            line = dict(
                color = 'rgb(204,102,102)',
                width = 2)))
# Trace of hem
Normal = go.Scatter3d(
        x = x[n_ALL:],
        y = y[n_ALL:],
        z = z[n_ALL:],
        opacity = 0.5,
        name = 'Normal',
        mode = 'markers',
        marker = dict(
            size = 5,
            color = 'rgb(102,153,255)',
            line = dict(
                color = 'rgb(000,102,204)',
                width = 2)))
data = [ALL, Normal]
layout = go.Layout(
        scene = dict(
        xaxis = dict(
                title='Cell Size'),
         yaxis = dict(
                title='Perimeter'),
        zaxis = dict(
                title='Haralick Difference Entropy'),),
        width=700,
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Top3features_scatter3D')

In [ ]:
# Reduce test data to the first 16 important features
RF_test_data = test_data[:,indices[:n_features]]

# Conversion of data from numpy array to dataframe
RF_test_data_df = pd.DataFrame.from_records(RF_test_data)
RF_test_data_df.columns = tuple(features[i] for i in indices[:n_features])

# insert labels for test data 
RF_test_data_df.insert(0,"Label",label_test,True)

# save train_data_df as a csv file
RF_test_data_df.to_csv('RF_test_data')
RF_test_data_df

# Dimension Reduction Using PCA


In [ ]:
# Dimension reduction using PCA
D = np.size(train_data[1])
pca = PCA(n_components = D)
pca.fit(train_data)  
PCA(copy=True, iterated_power='auto', n_components = D, random_state=None,
     svd_solver='auto', tol=0.0, whiten=False)
plt.plot(pca.explained_variance_ratio_) 
plt.title("Scree Plot")
plt.xlabel("Component Number")
plt.ylabel("Eigenvalue")
n_components = np.sum((pca.explained_variance_ratio_.cumsum()/pca.explained_variance_ratio_.sum()) < 0.99)+1
print('The number of principal components necessary to recover 99% of variance:',n_components)

In [ ]:
# choose number of components from the elbow test
n_components = 6
pca = PCA(n_components=n_components)
pca.fit(train_data)  
PCA_train_data = pca.fit_transform(train_data)
print('Shape of train data: ', np.shape(PCA_train_data))
PCA_test_data = pca.transform(test_data)
PCA_test_data

In [ ]:
# Conversion of data from numpy array to dataframe
PCA_train_data_df = pd.DataFrame.from_records(PCA_train_data)
PCA_train_data_df.columns = tuple('PC'+str(num+1) for num in range(np.size(PCA_train_data[1])))
PCA_train_data_df.insert(0,"Label",label_train,True)

# save train_data_df as a csv file
PCA_train_data_df.to_csv('Data/PCA_train_data.csv')
PCA_train_data_df

In [ ]:
# Visualizing 3-D numeric data with Scatter Plots
# PC1, PC2 and PC3
df = PCA_train_data_df.sample(frac=1)
fig = plt.figure(figsize=(16, 10))
ax = plt.axes(projection='3d')
xs = list(PCA_train_data_df['PC1'])
ys = list(PCA_train_data_df['PC2'])
zs = list(PCA_train_data_df['PC3'])
data_points = [(x, y, z) for x, y, z in zip(xs, ys, zs)]
colors = ['orange' if i == 1 else 'blue' for i in list(RF_train_data_df['Label'])]
for data, color in zip(data_points, colors):
    x, y, z = data
    ax.scatter(x, y, z,s=30, alpha=0.4, c=color, edgecolors='w')

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')

In [ ]:
# Conversion of data from numpy array to dataframe
PCA_test_data_df = pd.DataFrame.from_records(PCA_test_data)
PCA_test_data_df.columns = tuple('PC'+str(num+1) for num in range(np.size(PCA_test_data[1])))

# insert labels for test data 
PCA_test_data_df.insert(0,"Label",label_test,True)

# save test_data_df as a csv file
PCA_test_data_df.to_csv('Data/PCA_test_data.csv')
PCA_test_data_df

In [ ]:
## Visualization of top 3 principle components
# Trace of ALL
x = PCA_train_data_df['PC1']
y = PCA_train_data_df['PC2']
z = PCA_train_data_df['PC3']
ALL = go.Scatter3d(
        x = x[:n_ALL],
        y = y[:n_ALL],
        z = z[:n_ALL],
        opacity = 0.5,
        name = 'ALL',
        mode = 'markers',
        marker = dict(
            size = 5,
            color = 'rgb(255,102,102)',
            line = dict(
                color = 'rgb(204,102,102)',
                width = 2)))
# Trace of hem
Normal = go.Scatter3d(
        x = x[n_ALL:],
        y = y[n_ALL:],
        z = z[n_ALL:],
        opacity = 0.5,
        name = 'Normal',
        mode = 'markers',
        marker = dict(
            size = 5,
            color = 'rgb(102,153,255)',
            line = dict(
                color = 'rgb(000,102,204)',
                width = 2)))
data = [ALL, Normal]
layout = go.Layout(
        scene = dict(
        xaxis = dict(
                title='PC1'),
         yaxis = dict(
                title='PC2'),
        zaxis = dict(
                title='PC3'),),
        width=700,
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Top3PC_scatter3D')